In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Initialize Mediapipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Path for input videos
VIDEO_PATH = os.path.join('Video_Data')
DATA_PATH = os.path.join('MP_Data')  # Path to save extracted data

# Actions for classification
actions = ['abnormal', 'normal']

# Videos are processed into sequences of 30 frames
sequence_length = 30

# Function to extract keypoints using Mediapipe
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() \
        if results.face_landmarks else np.zeros(468 * 3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() \
        if results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() \
        if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() \
        if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([face, pose, lh, rh])

# Create directories for saving data
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)

# Process videos and save keypoints
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(VIDEO_PATH, action)
        for video_file in os.listdir(action_path):
            video_path = os.path.join(action_path, video_file)
            cap = cv2.VideoCapture(video_path)
            sequence = []
            frame_count = 0
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Resize frame for consistency
                frame = cv2.resize(frame, (640, 480))
                
                # Make detections
                image, results = frame.copy(), holistic.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                
                # Extract keypoints
                keypoints = extract_keypoints(results)
                sequence.append(keypoints)
                frame_count += 1

                # Stop after collecting the sequence length
                if frame_count == sequence_length:
                    break
            
            cap.release()
            
            # Save the sequence if it has the required length
            if len(sequence) == sequence_length:
                npy_path = os.path.join(DATA_PATH, action, f"{video_file.split('.')[0]}.npy")
                np.save(npy_path, np.array(sequence))

# Load the processed data
sequences, labels = [], []
label_map = {label: num for num, label in enumerate(actions)}

for action in actions:
    for file in os.listdir(os.path.join(DATA_PATH, action)):
        path = os.path.join(DATA_PATH, action, file)
        sequences.append(np.load(path))
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Dataset Summary
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (71, 30, 1662)
y_train shape: (71, 2)
X_test shape: (4, 30, 1662)
y_test shape: (4, 2)
